In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

from pyspark.sql.functions import col, current_date, when
from pyspark.sql.types import DateType

df_raw = (
    spark.read.option("header", True)
    .option("inferSchema", True)
    .csv("Files/scd_data.csv")
)

df_cleaned = df_raw.withColumn(
    "LoadDate",
    when(col("LoadDate").isNull(), current_date()).otherwise(col("LoadDate").cast(DateType()))
)

display(df_cleaned)


StatementMeta(, b650f8f3-9547-40a1-a58e-4c3be732c936, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 71ccb3bd-5671-4ceb-ae02-12fef213d6a7)

In [5]:
from pyspark.sql.functions import col, current_date, trim, lit
from pyspark.sql.types import DateType

df_casted = df_raw.withColumn(
    "LoadDate",
    trim(col("LoadDate")).cast(DateType())
)


df_deduped = df_casted.dropDuplicates()


df_final = df_deduped.withColumn(
    "LoadDate",
    when(col("LoadDate").isNull(), current_date()).otherwise(col("LoadDate"))
)

df_final.select("LoadDate").distinct().show()

StatementMeta(, b650f8f3-9547-40a1-a58e-4c3be732c936, 7, Finished, Available, Finished)

+----------+
|  LoadDate|
+----------+
|2023-06-01|
|2025-07-26|
|2024-01-01|
+----------+



In [6]:
df_addcolumn = df_final\
  .withColumn("StartDate", col("LoadDate"))\
  .withColumn("EndDate", lit(None).cast("date"))\
  .withColumn("IsActive", lit(True))

df_addcolumn

StatementMeta(, b650f8f3-9547-40a1-a58e-4c3be732c936, 8, Finished, Available, Finished)

DataFrame[EmpID: int, Name: string, Gender: string, JobTitle: string, Department: string, LoadDate: date, StartDate: date, EndDate: date, IsActive: boolean]

In [10]:
from pyspark.sql.functions import asc_nulls_last, row_number, lag
from pyspark.sql.window import Window

window_by = Window.partitionBy("EmpID").orderBy(asc_nulls_last("LoadDate"))
df_version = df_addcolumn.withColumn("rn", row_number().over(window_by))

df_changes = df_version.withColumn("PrevJobTitle", lag("JobTitle").over(window_by))\
    .withColumn("PrevDepartment", lag("Department").over(window_by))\
    .withColumn("IsChanged", (col("JobTitle") != col("PrevJobTitle")) | (col("Department") != col("PrevDepartment")))

df_changes.orderBy("EmpID", asc_nulls_last("LoadDate")).show()


StatementMeta(, b650f8f3-9547-40a1-a58e-4c3be732c936, 12, Finished, Available, Finished)

+-----+---------------+------+---------------+----------+----------+----------+-------+--------+---+---------------+--------------+---------+
|EmpID|           Name|Gender|       JobTitle|Department|  LoadDate| StartDate|EndDate|IsActive| rn|   PrevJobTitle|PrevDepartment|IsChanged|
+-----+---------------+------+---------------+----------+----------+----------+-------+--------+---+---------------+--------------+---------+
|    1|   Allison Hill|  Male|   BI Developer|        IT|2023-06-01|2023-06-01|   NULL|    true|  1|           NULL|          NULL|     NULL|
|    1|   Allison Hill|  Male|   Data Analyst|        HR|2025-07-26|2025-07-26|   NULL|    true|  2|   BI Developer|            IT|     true|
|    2|    Noah Rhodes|  Male|  Data Engineer|   Finance|2025-07-26|2025-07-26|   NULL|    true|  1|           NULL|          NULL|     NULL|
|    3|Angie Henderson|  Male|  Data Engineer|   Finance|2023-06-01|2023-06-01|   NULL|    true|  1|           NULL|          NULL|     NULL|
|    3

In [11]:
from pyspark.sql.functions import lead, date_sub


df_lead = df_changes.withColumn("NextLoadDate", lead("LoadDate").over(window_by))

df_scd = df_lead.withColumn("StartDate", col("LoadDate")) \
    .withColumn("EndDate", date_sub(col("NextLoadDate"), 1)) \
    .withColumn("IsActive", when(col("NextLoadDate").isNull(), lit(True)).otherwise(lit(False)))

df_scd = df_scd.select("EmpID", "Name", "Gender", "JobTitle", "Department",
                             "LoadDate", "StartDate", "EndDate", "IsActive",
                             "rn", "PrevJobTitle", "PrevDepartment", "IsChanged")

df_scd.orderBy("EmpID", "StartDate").show()

StatementMeta(, b650f8f3-9547-40a1-a58e-4c3be732c936, 13, Finished, Available, Finished)

+-----+---------------+------+---------------+----------+----------+----------+----------+--------+---+---------------+--------------+---------+
|EmpID|           Name|Gender|       JobTitle|Department|  LoadDate| StartDate|   EndDate|IsActive| rn|   PrevJobTitle|PrevDepartment|IsChanged|
+-----+---------------+------+---------------+----------+----------+----------+----------+--------+---+---------------+--------------+---------+
|    1|   Allison Hill|  Male|   BI Developer|        IT|2023-06-01|2023-06-01|2025-07-25|   false|  1|           NULL|          NULL|     NULL|
|    1|   Allison Hill|  Male|   Data Analyst|        HR|2025-07-26|2025-07-26|      NULL|    true|  2|   BI Developer|            IT|     true|
|    2|    Noah Rhodes|  Male|  Data Engineer|   Finance|2025-07-26|2025-07-26|      NULL|    true|  1|           NULL|          NULL|     NULL|
|    3|Angie Henderson|  Male|  Data Engineer|   Finance|2023-06-01|2023-06-01|2023-12-31|   false|  1|           NULL|          N

In [12]:
dim_employee = df_scd.select("EmpID", "Name", "Gender", "JobTitle", "Department", "LoadDate",
                                   "StartDate", "EndDate", "IsActive")

dim_employee.orderBy("EmpID", "StartDate").show(truncate=False)
dim_employee.write.mode("overwrite").saveAsTable("dim_employee")


StatementMeta(, b650f8f3-9547-40a1-a58e-4c3be732c936, 14, Finished, Available, Finished)

+-----+---------------+------+---------------+----------+----------+----------+----------+--------+
|EmpID|Name           |Gender|JobTitle       |Department|LoadDate  |StartDate |EndDate   |IsActive|
+-----+---------------+------+---------------+----------+----------+----------+----------+--------+
|1    |Allison Hill   |Male  |BI Developer   |IT        |2023-06-01|2023-06-01|2025-07-25|false   |
|1    |Allison Hill   |Male  |Data Analyst   |HR        |2025-07-26|2025-07-26|NULL      |true    |
|2    |Noah Rhodes    |Male  |Data Engineer  |Finance   |2025-07-26|2025-07-26|NULL      |true    |
|3    |Angie Henderson|Male  |Data Engineer  |Finance   |2023-06-01|2023-06-01|2023-12-31|false   |
|3    |Angie Henderson|Male  |ML Engineer    |IT        |2024-01-01|2024-01-01|2025-07-25|false   |
|3    |Angie Henderson|Male  |Product Manager|IT        |2025-07-26|2025-07-26|NULL      |true    |
|4    |Daniel Wagner  |Female|Data Analyst   |IT        |2025-07-26|2025-07-26|NULL      |true    |
